In [2]:
import sys
import warnings
import numpy as np
import pandas as pd
from data.data import process_data
from model import model
from keras.models import Model
from keras.callbacks import EarlyStopping
warnings.filterwarnings("ignore")


def train_model(model, X_train, y_train, name, config):
    """train
    train a single model.

    # Arguments
        model: Model, NN model to train.
        X_train: ndarray(number, lags), Input data for train.
        y_train: ndarray(number, ), result data for train.
        name: String, name of model.
        config: Dict, parameter for train.
    """

    model.compile(loss="mse", optimizer="rmsprop", metrics=['mape'])
    # early = EarlyStopping(monitor='val_loss', patience=30, verbose=0, mode='auto')
    hist = model.fit(
        X_train, y_train,
        batch_size=config["batch"],
        epochs=config["epochs"],
        validation_split=0.05)

    model.save('model/' + name + '.h5')
    df = pd.DataFrame.from_dict(hist.history)
    df.to_csv('model/' + name + ' loss.csv', encoding='utf-8', index=False)


def train_seas(models, X_train, y_train, name, config):
    """train
    train the SAEs model.

    # Arguments
        models: List, list of SAE model.
        X_train: ndarray(number, lags), Input data for train.
        y_train: ndarray(number, ), result data for train.
        name: String, name of model.
        config: Dict, parameter for train.
    """

    temp = X_train
    # early = EarlyStopping(monitor='val_loss', patience=30, verbose=0, mode='auto')

    for i in range(len(models) - 1):
        if i > 0:
            p = models[i - 1]
            hidden_layer_model = Model(input=p.input,
                                       output=p.get_layer('hidden').output)
            temp = hidden_layer_model.predict(temp)

        m = models[i]
        m.compile(loss="mse", optimizer="rmsprop", metrics=['mape'])

        m.fit(temp, y_train, batch_size=config["batch"],
              epochs=config["epochs"],
              validation_split=0.05)

        models[i] = m

    saes = models[-1]
    for i in range(len(models) - 1):
        weights = models[i].get_layer('hidden').get_weights()
        saes.get_layer('hidden%d' % (i + 1)).set_weights(weights)

    train_model(saes, X_train, y_train, name, config)


def main():
    lag = 12
   
    config = {"batch": 256, "epochs": 100}
    file1 = 'data/train.csv'
    file2 = 'data/test.csv'
    X_train, y_train, _, _, _ = process_data(file1, file2, lag)
    
    model_type = 'lstm'

    if model_type == 'lstm':
        X_train = np.reshape(X_train, (X_train.shape[0], X_train.shape[1], 1))
        m = model.get_lstm([12, 64, 64, 1])
        train_model(m, X_train, y_train, model_type, config)
    if model_type == 'gru':
        X_train = np.reshape(X_train, (X_train.shape[0], X_train.shape[1], 1))
        m = model.get_gru([12, 64, 64, 1])
        train_model(m, X_train, y_train, model_type, config)
    if model_type == 'saes':
        X_train = np.reshape(X_train, (X_train.shape[0], X_train.shape[1]))
        m = model.get_saes([12, 400, 400, 400, 1])
        train_seas(m, X_train, y_train, model_type, config)


if __name__ == '__main__':
    main()

Epoch 1/100
29/29 [==============================] - 9s 129ms/step - loss: 0.0395 - mape: 267442.0312 - val_loss: 0.0121 - val_mape: 127.0087
Epoch 2/100
29/29 [==============================] - 2s 63ms/step - loss: 0.0115 - mape: 92016.1016 - val_loss: 0.0105 - val_mape: 90.0114
Epoch 3/100
29/29 [==============================] - 2s 64ms/step - loss: 0.0103 - mape: 89475.1484 - val_loss: 0.0101 - val_mape: 80.7808
Epoch 4/100
29/29 [==============================] - 2s 76ms/step - loss: 0.0097 - mape: 89110.0859 - val_loss: 0.0101 - val_mape: 98.1125
Epoch 5/100
29/29 [==============================] - 2s 65ms/step - loss: 0.0091 - mape: 82294.6875 - val_loss: 0.0094 - val_mape: 91.1464
Epoch 6/100
29/29 [==============================] - 2s 58ms/step - loss: 0.0086 - mape: 87338.0938 - val_loss: 0.0081 - val_mape: 72.7311
Epoch 7/100
29/29 [==============================] - 2s 66ms/step - loss: 0.0080 - mape: 84812.7031 - val_loss: 0.0075 - val_mape: 69.1064
Epoch 8/100
29/29 [=====

In [3]:
import sys
import warnings
import numpy as np
import pandas as pd
from data.data import process_data
from model import model
from keras.models import Model
from keras.callbacks import EarlyStopping
warnings.filterwarnings("ignore")


def train_model(model, X_train, y_train, name, config):
    """train
    train a single model.

    # Arguments
        model: Model, NN model to train.
        X_train: ndarray(number, lags), Input data for train.
        y_train: ndarray(number, ), result data for train.
        name: String, name of model.
        config: Dict, parameter for train.
    """

    model.compile(loss="mse", optimizer="rmsprop", metrics=['mape'])
    # early = EarlyStopping(monitor='val_loss', patience=30, verbose=0, mode='auto')
    hist = model.fit(
        X_train, y_train,
        batch_size=config["batch"],
        epochs=config["epochs"],
        validation_split=0.05)

    model.save('model/' + name + '.h5')
    df = pd.DataFrame.from_dict(hist.history)
    df.to_csv('model/' + name + ' loss.csv', encoding='utf-8', index=False)


def train_seas(models, X_train, y_train, name, config):
    """train
    train the SAEs model.

    # Arguments
        models: List, list of SAE model.
        X_train: ndarray(number, lags), Input data for train.
        y_train: ndarray(number, ), result data for train.
        name: String, name of model.
        config: Dict, parameter for train.
    """

    temp = X_train
    # early = EarlyStopping(monitor='val_loss', patience=30, verbose=0, mode='auto')

    for i in range(len(models) - 1):
        if i > 0:
            p = models[i - 1]
            hidden_layer_model = Model(input=p.input,
                                       output=p.get_layer('hidden').output)
            temp = hidden_layer_model.predict(temp)

        m = models[i]
        m.compile(loss="mse", optimizer="rmsprop", metrics=['mape'])

        m.fit(temp, y_train, batch_size=config["batch"],
              epochs=config["epochs"],
              validation_split=0.05)

        models[i] = m

    saes = models[-1]
    for i in range(len(models) - 1):
        weights = models[i].get_layer('hidden').get_weights()
        saes.get_layer('hidden%d' % (i + 1)).set_weights(weights)

    train_model(saes, X_train, y_train, name, config)


def main():
    lag = 12
   
    config = {"batch": 256, "epochs": 100}
    file1 = 'data/train.csv'
    file2 = 'data/test.csv'
    X_train, y_train, _, _, _ = process_data(file1, file2, lag)
    
    model_type = 'saes'

    if model_type == 'gru':
        X_train = np.reshape(X_train, (X_train.shape[0], X_train.shape[1], 1))
        m = model.get_lstm([12, 64, 64, 1])
        train_model(m, X_train, y_train, model_type, config)
    if model_type == 'gru':
        X_train = np.reshape(X_train, (X_train.shape[0], X_train.shape[1], 1))
        m = model.get_gru([12, 64, 64, 1])
        train_model(m, X_train, y_train, model_type, config)
    if model_type == 'saes':
        X_train = np.reshape(X_train, (X_train.shape[0], X_train.shape[1]))
        m = model.get_saes([12, 400, 400, 400, 1])
        train_seas(m, X_train, y_train, model_type, config)


if __name__ == '__main__':
    main()

Epoch 1/100
29/29 [==============================] - 8s 87ms/step - loss: 0.0344 - mape: 207715.4688 - val_loss: 0.0079 - val_mape: 359149.1562
Epoch 2/100
29/29 [==============================] - 1s 49ms/step - loss: 0.0076 - mape: 76531.3125 - val_loss: 0.0060 - val_mape: 253342.5781
Epoch 3/100
29/29 [==============================] - 1s 50ms/step - loss: 0.0070 - mape: 72865.0625 - val_loss: 0.0061 - val_mape: 259175.6562
Epoch 4/100
29/29 [==============================] - 1s 49ms/step - loss: 0.0065 - mape: 62703.9531 - val_loss: 0.0055 - val_mape: 250613.3438
Epoch 5/100
29/29 [==============================] - 1s 50ms/step - loss: 0.0062 - mape: 61063.7188 - val_loss: 0.0050 - val_mape: 242428.3125
Epoch 6/100
29/29 [==============================] - 1s 50ms/step - loss: 0.0059 - mape: 65083.2695 - val_loss: 0.0048 - val_mape: 233459.5312
Epoch 7/100
29/29 [==============================] - 1s 50ms/step - loss: 0.0056 - mape: 63037.4102 - val_loss: 0.0045 - val_mape: 234826.390

Epoch 58/100
29/29 [==============================] - 2s 53ms/step - loss: 0.0033 - mape: 37773.8789 - val_loss: 0.0029 - val_mape: 151376.7344
Epoch 59/100
29/29 [==============================] - 2s 60ms/step - loss: 0.0033 - mape: 41605.4141 - val_loss: 0.0030 - val_mape: 152672.9531
Epoch 60/100
29/29 [==============================] - 2s 56ms/step - loss: 0.0032 - mape: 39773.4570 - val_loss: 0.0033 - val_mape: 131311.0469
Epoch 61/100
29/29 [==============================] - 2s 53ms/step - loss: 0.0032 - mape: 40677.7266 - val_loss: 0.0030 - val_mape: 133344.9375
Epoch 62/100
29/29 [==============================] - 1s 47ms/step - loss: 0.0033 - mape: 34837.8281 - val_loss: 0.0033 - val_mape: 129386.2969
Epoch 63/100
29/29 [==============================] - 1s 49ms/step - loss: 0.0032 - mape: 46368.5547 - val_loss: 0.0031 - val_mape: 132512.9375
Epoch 64/100
29/29 [==============================] - 1s 48ms/step - loss: 0.0032 - mape: 40274.5352 - val_loss: 0.0029 - val_mape: 1429

In [4]:
import sys
import warnings
import numpy as np
import pandas as pd
from data.data import process_data
from model import model
from keras.models import Model
from keras.callbacks import EarlyStopping
warnings.filterwarnings("ignore")


def train_model(model, X_train, y_train, name, config):
    """train
    train a single model.

    # Arguments
        model: Model, NN model to train.
        X_train: ndarray(number, lags), Input data for train.
        y_train: ndarray(number, ), result data for train.
        name: String, name of model.
        config: Dict, parameter for train.
    """

    model.compile(loss="mse", optimizer="rmsprop", metrics=['mape'])
    # early = EarlyStopping(monitor='val_loss', patience=30, verbose=0, mode='auto')
    hist = model.fit(
        X_train, y_train,
        batch_size=config["batch"],
        epochs=config["epochs"],
        validation_split=0.05)

    model.save('model/' + name + '.h5')
    df = pd.DataFrame.from_dict(hist.history)
    df.to_csv('model/' + name + ' loss.csv', encoding='utf-8', index=False)


def train_seas(models, X_train, y_train, name, config):
    """train
    train the SAEs model.

    # Arguments
        models: List, list of SAE model.
        X_train: ndarray(number, lags), Input data for train.
        y_train: ndarray(number, ), result data for train.
        name: String, name of model.
        config: Dict, parameter for train.
    """

    temp = X_train
    # early = EarlyStopping(monitor='val_loss', patience=30, verbose=0, mode='auto')

    for i in range(len(models) - 1):
        if i > 0:
            p = models[i - 1]
            hidden_layer_model = Model(input=p.input,
                                       output=p.get_layer('hidden').output)
            temp = hidden_layer_model.predict(temp)

        m = models[i]
        m.compile(loss="mse", optimizer="rmsprop", metrics=['mape'])

        m.fit(temp, y_train, batch_size=config["batch"],
              epochs=config["epochs"],
              validation_split=0.05)

        models[i] = m

    saes = models[-1]
    for i in range(len(models) - 1):
        weights = models[i].get_layer('hidden').get_weights()
        saes.get_layer('hidden%d' % (i + 1)).set_weights(weights)

    train_model(saes, X_train, y_train, name, config)


def main():
    lag = 12
   
    config = {"batch": 256, "epochs": 100}
    file1 = 'data/train.csv'
    file2 = 'data/test.csv'
    X_train, y_train, _, _, _ = process_data(file1, file2, lag)
    
    model_type = 'saes'

    if model_type == 'lstm':
        X_train = np.reshape(X_train, (X_train.shape[0], X_train.shape[1], 1))
        m = model.get_lstm([12, 64, 64, 1])
        train_model(m, X_train, y_train, model_type, config)
    if model_type == 'gru':
        X_train = np.reshape(X_train, (X_train.shape[0], X_train.shape[1], 1))
        m = model.get_gru([12, 64, 64, 1])
        train_model(m, X_train, y_train, model_type, config)
    if model_type == 'saes':
        X_train = np.reshape(X_train, (X_train.shape[0], X_train.shape[1]))
        m = model.get_saes([12, 400, 400, 400, 1])
        train_seas(m, X_train, y_train, model_type, config)


if __name__ == '__main__':
    main()

Epoch 1/100
29/29 [==============================] - 1s 18ms/step - loss: 0.0383 - mape: 226800.8906 - val_loss: 0.0244 - val_mape: 654131.1250
Epoch 2/100
29/29 [==============================] - 0s 10ms/step - loss: 0.0250 - mape: 161223.0156 - val_loss: 0.0157 - val_mape: 557423.0000
Epoch 3/100
29/29 [==============================] - 0s 8ms/step - loss: 0.0187 - mape: 113056.5781 - val_loss: 0.0120 - val_mape: 473960.8750
Epoch 4/100
29/29 [==============================] - 0s 10ms/step - loss: 0.0155 - mape: 88586.8438 - val_loss: 0.0107 - val_mape: 339571.4375
Epoch 5/100
29/29 [==============================] - 0s 8ms/step - loss: 0.0141 - mape: 83853.0703 - val_loss: 0.0091 - val_mape: 341704.7500
Epoch 6/100
29/29 [==============================] - 0s 8ms/step - loss: 0.0128 - mape: 100438.2812 - val_loss: 0.0089 - val_mape: 334263.5938
Epoch 7/100
29/29 [==============================] - 0s 9ms/step - loss: 0.0129 - mape: 62413.5898 - val_loss: 0.0089 - val_mape: 268087.5938

TypeError: ('Keyword argument not understood:', 'input')